In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import ImageFolder
import numpy as np
import os
from tqdm import tqdm
import wandb

In [18]:
!wandb login 6001619563748a57b4114b0bb090fd4129ba6122

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [3]:
# Configuration
config = {
    'batch_size': 16,
    'epochs': 10,
    'learning_rate': 3e-5,  # Lower learning rate for full model training
    'weight_decay': 1e-4,
    'dropout_rate': 0.5,
    'classifier_hidden_units': 512,
    'scheduler_factor': 0.1,
    'scheduler_patience': 2,
    'model_architecture': 'resnet50',
    'pretrained': True,
    'optimizer': 'AdamW',
    'loss_function': 'CrossEntropyLoss'
}


In [4]:
wandb.init(project="inaturalist-classification", config=config)


wandb: Currently logged in as: cs24m025 (manglesh-patidar-cs24m025) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [5]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [6]:
# Data transformations (same as before)
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [7]:
def get_dataset_and_loaders(data_dir, train_transform, val_transform, batch_size=32, val_split=0.2):
    full_dataset = ImageFolder(os.path.join(data_dir, 'train'), transform=train_transform)
    
    # Stratified split
    targets = np.array(full_dataset.targets)
    train_indices, val_indices = [], []
    
    for class_idx in np.unique(targets):
        class_indices = np.where(targets == class_idx)[0]
        n_val = int(len(class_indices) * val_split)
        np.random.shuffle(class_indices)
        val_indices.extend(class_indices[:n_val])
        train_indices.extend(class_indices[n_val:])
    
    train_dataset = Subset(full_dataset, train_indices)
    val_dataset = Subset(full_dataset, val_indices)
    
    val_dataset.dataset.transform = val_transform
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
    
    return train_loader, val_loader, full_dataset.classes

In [8]:

# Model definition for full training
def create_model(num_classes):
    model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
    
    # Replace classifier (all parameters remain trainable)
    model.fc = nn.Sequential(
        nn.Dropout(config['dropout_rate']),
        nn.Linear(2048, config['classifier_hidden_units']),
        nn.ReLU(),
        nn.Linear(config['classifier_hidden_units'], num_classes)
    )
    
    return model.to(device)

In [9]:
# Initialize dataset and model
train_loader, val_loader, classes = get_dataset_and_loaders(
    '/kaggle/input/inaturalist/inaturalist_12K',
    train_transform,
    val_transform,
    batch_size=config['batch_size']
)


In [10]:

model = create_model(num_classes=len(classes))



Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 211MB/s]


In [11]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(
    model.parameters(),  # Train all parameters
    lr=config['learning_rate'],
    weight_decay=config['weight_decay']
)



In [12]:
# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='max',
    factor=config['scheduler_factor'],
    patience=config['scheduler_patience']
)

# Wandb monitoring
wandb.watch(model, log_freq=100, log="all")

In [13]:


# Training loop
best_val_acc = 0.0
for epoch in range(config['epochs']):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1} [Train]"):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    train_loss = train_loss / len(train_loader.dataset)
    train_acc = correct / total
    
    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc=f"Epoch {epoch+1} [Val]"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    val_loss = val_loss / len(val_loader.dataset)
    val_acc = correct / total
    
    # Update scheduler
    scheduler.step(val_acc)
    
    # Log metrics
    wandb.log({
        "epoch": epoch,
        "train_loss": train_loss,
        "train_accuracy": train_acc,
        "val_loss": val_loss,
        "val_accuracy": val_acc,
        "learning_rate": optimizer.param_groups[0]['lr']
    })
    
    print(f"Epoch {epoch+1}/{config['epochs']}:")
    print(f"Train Loss: {train_loss:.4f} | Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f} | Acc: {val_acc:.4f}")
    print(f"LR: {optimizer.param_groups[0]['lr']:.2e}")
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_full_model.pth')
        wandb.save('best_full_model.pth')
        print(f"New best model saved with val acc: {best_val_acc:.4f}")
        wandb.run.summary["best_val_accuracy"] = best_val_acc

print(f"Training complete. Best validation accuracy: {best_val_acc:.4f}")
wandb.finish()

Epoch 1 [Val]: 100%|██████████| 125/125 [00:13<00:00,  9.11it/s]


Epoch 1/10:
Train Loss: 1.3193 | Acc: 0.6168
Val Loss: 0.5305 | Acc: 0.8414
LR: 3.00e-05
New best model saved with val acc: 0.8414


Epoch 2 [Val]: 100%|██████████| 125/125 [00:11<00:00, 11.05it/s]


Epoch 2/10:
Train Loss: 0.4823 | Acc: 0.8508
Val Loss: 0.4265 | Acc: 0.8634
LR: 3.00e-05
New best model saved with val acc: 0.8634


Epoch 3 [Val]: 100%|██████████| 125/125 [00:11<00:00, 11.07it/s]


...

Epoch 9 [Val]: 100%|██████████| 125/125 [00:11<00:00, 11.23it/s]


Epoch 9/10:
Train Loss: 0.0312 | Acc: 0.9920
Val Loss: 0.5110 | Acc: 0.8759
LR: 3.00e-06


Epoch 10 [Val]: 100%|██████████| 125/125 [00:11<00:00, 10.70it/s]

Epoch 10/10:
Train Loss: 0.0255 | Acc: 0.9948
Val Loss: 0.4984 | Acc: 0.8784
LR: 3.00e-07
Training complete. Best validation accuracy: 0.8824


epoch,▁▂▃▃▄▅▆▆▇█
learning_rate,██████▂▂▂▁
train_accuracy,▁▅▆▇██████
train_loss,█▃▂▂▁▁▁▁▁▁
val_accuracy,▁▅▅█▇▇▇▇▇▇
val_loss,█▂▁▁▄▄▆▇▇▆
best_val_accuracy,0.88244
epoch,9
learning_rate,0.0
train_accuracy,0.99475
train_loss,0.02552


In [19]:
wandb.init(project="inaturalist-classification", config=config)

In [21]:
# Load the best model for testing
best_model = create_model(num_classes=len(classes))
best_model.load_state_dict(torch.load('best_full_model.pth'))
best_model.to(device)
best_model.eval()

# Test data transformations (should match validation transforms)
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load test dataset
test_dataset = ImageFolder(
    os.path.join('/kaggle/input/inaturalist/inaturalist_12K', 'val'),  # or 'test' if available
    transform=test_transform
)

test_loader = DataLoader(
    test_dataset,
    batch_size=config['batch_size'],
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

# Test evaluation
test_loss = 0.0
correct = 0
total = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Testing"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = best_model(inputs)
        loss = criterion(outputs, labels)
        
        test_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_loss = test_loss / len(test_loader.dataset)
test_acc = correct / total

# Log test results to wandb
wandb.log({
    "test_loss": test_loss,
    "test_accuracy": test_acc
})

# Print test results
print(f"\nTest Results:")
print(f"Loss: {test_loss:.4f} | Accuracy: {test_acc:.4f}")

# Log confusion matrix
wandb.log({
    "confusion_matrix": wandb.plot.confusion_matrix(
        probs=None,
        y_true=all_labels,
        preds=all_preds,
        class_names=classes)
})

# Optionally: Log per-class metrics
from sklearn.metrics import classification_report
print("\nClassification Report:")
report = classification_report(all_labels, all_preds, target_names=classes, output_dict=True)
wandb.log({"classification_report": report})
print(classification_report(all_labels, all_preds, target_names=classes))

wandb.finish()

/tmp/ipykernel_31/1231587906.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load('best_full_model.pth'))
Testing: 100%|██████████| 125/


Test Results:
Loss: 0.3897 | Accuracy: 0.8805

Classification Report:
              precision    recall  f1-score   support

    Amphibia       0.92      0.92      0.92       200
    Animalia       0.89      0.85      0.87       200
   Arachnida       0.91      0.92      0.91       200
        Aves       0.96      0.91      0.93       200
       Fungi       0.86      0.92      0.88       200
     Insecta       0.90      0.84      0.87       200
    Mammalia       0.83      0.90      0.86       200
    Mollusca       0.85      0.83      0.84       200
     Plantae       0.85      0.83      0.84       200
    Reptilia       0.86      0.90      0.88       200

    accuracy                           0.88      2000
   macro avg       0.88      0.88      0.88      2000
weighted avg       0.88      0.88      0.88      2000



test_accuracy,▁
test_loss,▁
test_accuracy,0.8805
test_loss,0.38968
